# SVM

In [ ]:
import tensorflow as tf
import datetime, os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import svm
from datetime import datetime
from scipy.stats import mode
from scipy.stats import norm

In [ ]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

SVM viene de Support Vector Machines, es un algoritmo diferente a la regression logistica dado, que aunque se basa en las mismas ideas.
Agrega una seccion de confianza que no tiene la regresion logistica. Y eso muchas veces lo hace mas robusto y capaz de generalizar mejor el aprendizaje.

### SVM 

Es un algoritmo de aprendizaje supervisado creado por Vladimir Vapnik junto a otros colegas en el AT&T Bell Laboratories. Y es un algoritmo que es ampliamente popular en aplicaciones biologicas.
Es usado para clasificacion, regresion o detectar outliers. 


#### Ventajas
- Efectivo en espcaios de muchas dimensiones
- Se mantiene efectivo en casos donde el numero de dimensiones es mayor al numero de muestras.
- Usa un subset de puntos de entrenamiento en la funcion de decision que utiliza de soporte. (Support Vectors). 
- Versatil, ya que se puede customizar el kernel para la funcion de decision.


#### Desventajas
- Es posible el sobreajuste si el numero de caracteristicas es mucho mayor que el numero de muestras. Dependera de la decision de los hyperparametros para evitarlo.
- No calcula estimaciones de probabildad directamente, sino calcula un 5 Fold CV costoso.


Ejemplo de lineas de decision en regresion logistica:
<figure>
<img id="img1" src="https://miro.medium.com/max/600/0*9jEWNXTAao7phK-5.png">
</figure>

Ejemplo de una linea de decision optima (SVM):
<figure>
<img id="img1" src="https://miro.medium.com/max/600/0*0o8xIA4k3gXUDCFU.png">
</figure>


Los modelos SVM se consideran algoritmos de aprendizaje robustos dado ha estas lineas de soporte que ayudan a encontrar una barrera de decision optima. El plano de separacion entre clases que genera, le otorga la robustez que le caracteriza.

SVM  se basa en que la predicion ya no tomara valores de 0 o 1. Sino cambia estos valores por -1 y 1. Ademas la funcion de costo tambien cambia dado en parte a este cambio. Pero la hipotesis por lo tanto tomara la forma siguiente: 
$$h_{w,b}(x) = g(w^{T} x + b)$$

Notese que sigue siendo una hipotesis similar a la regresion logisitca. La diferencia ahora es que se ha aumentado el margen para separar las clases. Lo que se busca ahora es que *g(z) = 1* cuando *z* que es igual a  $w^{T} x + b)$ sea mucho mayor a 1. La frontera de decision ya no es solamente una linea. Por lo tanto *g(z) = 1* cuando *z* sea negativo. 

Nuestra funcion de costo entoces buscara generar mayor penalizacion para los casos en los que si $y = 1$ y $w^{T} x + b) < 0$ y cuando  $y = 0$ pero $w^{T} x + b) > 0$

### Funcion de costo
La funcion de costo es parecida en forma a la regresio logistica, pero ahora la funcion de SVM tiene forma de bisagra. Es por eso que le dicen hinge. 
Ahora para SVM se utilizara la siguiente: 

$$  C * \sum_{i=1}^{n}\max(0,1-y_i(w^{T}x_i + b)) + \frac{1}{2}\left\|w\right\|^2$$

Tambien se utiliza un parametro diferente para la regularizacion. En regresion logistica normalmente se aplicaba a los parametros segun fuera el metodo (L2 o L1), y es el hyperparametro denotado por lambda. 

$$A + \lambda * B$$

En SVM se utilziara el hyperparametro `C`:
$$ C * A + B $$
Es una forma de manejar el trade off del sesgo y varianza tambien.

### Kernel Tricks
La funcion Kernel de el algoritmo de SVM es una funcion que nos permite transformar los datos de entrada (caracteristicas) moviendolo a otro espacio y encontrando hyperplanos que separen de mejor manera los datos. La idea es encontrar una funcion que permita comparar que tan similares son dos vectores. Si dos vectores son similares el producto punto entre los vectores se hara mas grande, lo que precisamente buscamos por lo tanto es una funcion que separe nuestras observaciones al tomar un valor grande. 

SVM se dice ser flexible gracias a la capacidad de encontrar mejores barreras de decision gracias a que se puede customizar por medio de la funcion kernel. 
Existen Kernels ya definidos como los que ya trae la libreria de Scikit Learn: 

- linear: $\langle x, x'\rangle.$
- polynomial: $(\gamma \langle x, x'\rangle + r)^d$
- rbf: $\exp(-\gamma \|x-x'\|^2)$
- sigmoid $\tanh(\gamma \langle x,x'\rangle + r)$

Pero la funcion de Kernel permite customizar el algoritmo SVM, y esto hace posible encontrar nuevos planos que separen de mejor manera los datos.

### Conclusiones
- A diferencia de otros algoritmos de clasificacion que tienen como salida una probabilidad, Support Vector Machine no tiene de salida una probabilidad. Si no tiene como resultado la clase directamente. 
- SVM tuvo una gran aceptacion e interes en aplicaciones medicas antes de la tendencia actual de Deep Learning, y la razon es su buen margen de confianza aun aplicado a espacios de dimensiones mayores.
- Sino se necesita una probabilidad o nivel de certeza, es una buena opcion para aplicar en cualquier tipo de deataset gracias a su flexibilidad con los kernel tricks


### Bibliografia
[Scikit Learn SVM](https://scikit-learn.org/stable/modules/svm.html)

### Ejemplo